# Tarea5: Series de Taylor

Fecha de envío del PR inicial: **viernes 21 de abril**

Fecha de aceptación del PR: **martes 25 de abril, antes de la clase**

---

## Ejercicio 1

Implementen una nueva estructura paramétrica (`type`) que defina el tipo `Taylor`, donde el parámetro debe ser un subtipo de `Number`. Definan métodos que implementen las operaciones aritméticas básicas (`+`, `-`, `*`, `/`) y la igualdad (`==`). Esto deberá ser incluido en un módulo.

Incluyan pruebas (en el archivo "runtests_taylor.jl") para cada uno de los métodos que implementen.


In [1]:
"""
Taylor

Definición de polinomios de Taylor, donde v representa los coeficientes 
\$f_{[k]} = \\frac{f^{(k)}(x_0)}{k!} = \\frac{1}{k!}\\frac{d^k f}{dx^k}(x_0)\$ evaluados en \$x_0\$ y n es 
el grado del polinomio.
"""
type Taylor{T<:Number}
    v::Array{T,1}
    n::Int
end
function Taylor{T<:Number}(v::T)#función Taylor aplicada sobre un número
    x=[zero(eltype(v))]
    x[1]=v
    return Taylor(x,0)
end
function Taylor{T<:Number}(v::Array{T,1})#función Taylor aplicada sobre un vector
    n=length(v)
    return Taylor(v,n-1)
end

Taylor{T<:Number}

In [2]:
?Taylor

search:



Taylor

Definición de polinomios de Taylor, donde v representa los coeficientes  $f_{[k]} = \frac{f^{(k)}(x_0)}{k!} = \frac{1}{k!}\frac{d^k f}{dx^k}(x_0)$ evaluados en $x_0$ y n es  el grado del polinomio.


**Operaciones aritméticas para coeficientes de Taylor**
\begin{eqnarray}
(f+g)_{[k]} & = & f_{[k]} + g_{[k]} ,\\
(f-g)_{[k]} & = & f_{[k]} - g_{[k]} ,\\
(f \cdot g)_{[k]} & = & \sum_{i=0}^k f_{[i]} \,g_{[k-i]} \, ,\\
\Big(\frac{f}{g}\Big)_{[k]} & = & \frac{1}{g_{[0]}}
\Big( f_{[k]} - \sum_{i=0}^{k-1} \big(\frac{f}{g}\big)_{[i]} \, g_{[k-i]} \Big) . \\
\end{eqnarray}

In [3]:
function igualvec{T<:Number}(A::Array{T,1},n::T)#vector A con n elementos
    a=[i=zero(eltype(A)) for i=1:n]
    if n<length(A)
        for i in 1:n
            a[i]=A[i]
        end
        return a
    elseif n==length(A)
        return A
    else
        for i in 1:length(A)
            a[i]=A[i]
        end
        return a
    end
end
function igualvec{T<:Number}(a::Array{T,1},b::Array{T,1}) #función que iguala la longitud de los vectores. Se llenan con ceros
    if length(a)<length(b)
        x=[i=zero(eltype(a)) for i=1:length(b)]
        for i in 1:length(a)
            x[i]=a[i]
        end
        return x,b
    elseif length(a)==length(b)
        return a,b
    else 
        x=[i=zero(eltype(b)) for i=1:length(a)]
        for i in 1:length(b)
            x[i]=b[i]
        end
        return a,x
    end
end
function taylorsum(a,b)#función que suma los coeficientes tipo Taylor
    (x,y)=igualvec(a,b)
    if length(a)<length(b)
        return Taylor(x+y,length(b)-1)
    elseif length(a)==length(b)
        return Taylor(a+b,length(b)-1)
    else
        return Taylor(x+y,length(a)-1)
    end
end
function taylorres(a,b)#función que resta los coeficientes tipo Taylor
    (x,y)=igualvec(a,b)
    if length(a)<length(b)
        return Taylor(x-y,length(b)-1)
    elseif length(a)==length(b)
        return Taylor(a-b,length(b)-1)
    else
        return Taylor(x-y,length(a)-1)
    end
end
function producto(a,b)#producto de los coeficientes tipo Taylor para dos vectores de igual número de elementos
    ab=[i=zero(eltype(a)) for i=1:length(a)]
    for k in 1:length(a)
        for i in 1:k
            ab[k]+=a[i]*b[k+1-i]
        end
    end
    return ab
end
function taylorprod(a,b)#función que devuelve el producto de los coeficientes tipo Taylor
    (x,y)=igualvec(a,b)
    if length(a)<length(b)
        return Taylor(producto(x,y),length(b)-1)
    elseif length(a)==length(b)
        return Taylor(producto(a,b),length(b)-1)
    else
        return Taylor(producto(x,y),length(a)-1)
    end
end
function division(a,b)#división de los coeficientes tipo Taylor para dos vectores de igual número de elementos 
    if b[1]!=zero(eltype(b))
        divab=[i=zero(eltype(a)) for i=1:length(a)]
        divab[1]=a[1]/b[1]
        for k in 2:length(a)
            l=zero(eltype(a))
            for i in 1:k-1
                l+=divab[i]*b[k-i+1]
            end
            divab[k]=(1/b[1])*(a[k]-l)
        end
        return divab
    else
        return error("b[1] debe ser distinto de cero")
    end
end
function taylordiv(a,b)#función que devuelve la división de los coeficientes tipo Taylor
    (x,y)=igualvec(a,b)
    if length(a)<length(b)
        return Taylor(division(x,y),length(b)-1)
    elseif length(a)==length(b)
        return Taylor(division(a,b),length(b)-1)
    else
        return Taylor(division(x,y),length(a)-1)
    end
end

taylordiv (generic function with 1 method)

In [4]:
import Base:+,-,*,/,==
for (f,f1,f2,f3) = ((:+,Taylor,Taylor,(:(taylorsum(a.v,b.v)))),
                    (:+,Taylor,Number,(:(Taylor(a.v)+Taylor(b)))),
                    (:+,Number,Taylor,(:(Taylor(a)+Taylor(b.v)))),
                    (:-,Taylor,Taylor,(:(taylorres(a.v,b.v)))),
                    (:-,Taylor,Number,(:(Taylor(a.v)-Taylor(b)))),
                    (:-,Number,Taylor,(:(Taylor(a)-Taylor(b.v)))),
                    (:*,Taylor,Taylor,(:(taylorprod(a.v,b.v)))),
                    (:*,Taylor,Number,(:(Taylor(a.v)*Taylor(b)))),
                    (:*,Number,Taylor,(:(Taylor(a)*Taylor(b.v)))),
                    (:/,Taylor,Taylor,(:(taylordiv(a.v,b.v)))),
                    (:/,Taylor,Number,(:(Taylor(a.v)/Taylor(b)))),
                    (:/,Number,Taylor,(:(Taylor(a)/Taylor(b.v)))),
                    (:(==),Taylor,Taylor,(:(a.v==b.v && a.n==b.n))),
                    (:(==),Taylor,Number,(:(Taylor(a)==Taylor(b)))),
                    (:(==),Number,Taylor,(:(Taylor(a)==Taylor(b)))),)
              
    ex = quote
        $f(a::$f1,b::$f2)=$f3
    end
    @eval $ex
end


In [5]:
a=[2.,3,4,5,6]
b=[1.,1,3]
c=[1.,1,3,0,0]
x=Taylor(a)
y=Taylor(b)
z=Taylor(1.0)
h=[2.,0,0]
using Base.Test
@testset "pruebas con operaciones entre tipos Taylor"begin
    @test z==Taylor([1],0)
    @test x==Taylor([2,3,4,5,6],4)
    @test y==Taylor([1,1,3],2)
    @test z==Taylor([1],0)
    @test x+y==Taylor([3,4,7,5,6],4)
    @test 1+y==Taylor([2,1,3],2)
    @test y+1==Taylor([2,1,3],2)
    @test x-y==Taylor([1,2,1,5,6],4)
    @test y-x==Taylor([-1,-2,-1,-5,-6],4)
    @test 1-y==Taylor([0,-1,-3],2)
    @test y-1==Taylor([0,1,3],2)
    @test y*x==Taylor(producto(c,a),4)
    @test 2*y==Taylor([2,2,6],2)
    @test y*3==Taylor([3,3,9],2)
    @test y/x==Taylor(division(c,a),4)
    @test 2/y==Taylor(division(h,b),2)
    @test y/3==Taylor([1/3,1/3,1],2)
end

Test Summary:                              | Pass  Total
  pruebas con operaciones entre tipos Taylor |   17     17


Base.Test.DefaultTestSet("pruebas con operaciones entre tipos Taylor",Any[],17,false)

In [47]:
workspace()
__precompile__(true)

module AD
    import Base: +, -, *, /, ==
    export Taylor,producto,division,igualvec
"""
Taylor

Definición de polinomios de Taylor, donde v representa los coeficientes 
\$f_{[k]} = \\frac{f^{(k)}(x_0)}{k!} = \\frac{1}{k!}\\frac{d^k f}{dx^k}(x_0)\$ evaluados en \$x_0\$ y n es 
el grado del polinomio.
"""
type Taylor{T<:Number}
    v::Array{T,1}
    n::Int
end
function Taylor{T<:Number}(v::T)#función Taylor aplicada sobre un número
    x=[zero(eltype(v))]
    x[1]=v
    return Taylor(x,0)
end
function Taylor{T<:Number}(v::Array{T,1})#función Taylor aplicada sobre un vector
    n=length(v)
    return Taylor(v,n-1)
end
function igualvec(a,b) #función que iguala la longitud de los vectores. Se llenan con ceros
    if length(a)<length(b)
        x=[i=zero(eltype(a)) for i=1:length(b)]
        for i in 1:length(a)
            x[i]=a[i]
        end
        return x,b
    elseif length(a)==length(b)
        return a,b
    else 
        x=[i=zero(eltype(b)) for i=1:length(a)]
        for i in 1:length(b)
            x[i]=b[i]
        end
        return a,x
    end
end
function taylorsum(a,b)#función que suma los coeficientes tipo Taylor
    (x,y)=igualvec(a,b)
    if length(a)<length(b)
        return Taylor(x+y,length(b)-1)
    elseif length(a)==length(b)
        return Taylor(a+b,length(b)-1)
    else
        return Taylor(x+y,length(a)-1)
    end
end
function taylorres(a,b)#función que resta los coeficientes tipo Taylor
    (x,y)=igualvec(a,b)
    if length(a)<length(b)
        return Taylor(x-y,length(b)-1)
    elseif length(a)==length(b)
        return Taylor(a-b,length(b)-1)
    else
        return Taylor(x-y,length(a)-1)
    end
end
function producto(a,b)#producto de los coeficientes tipo Taylor para dos vectores de igual número de elementos
    ab=[i=zero(eltype(a)) for i=1:length(a)]
    for k in 1:length(a)
        for i in 1:k
            ab[k]+=a[i]*b[k+1-i]
        end
    end
    return ab
end
function taylorprod(a,b)#función que devuelve el producto de los coeficientes tipo Taylor
    (x,y)=igualvec(a,b)
    if length(a)<length(b)
        return Taylor(producto(x,y),length(b)-1)
    elseif length(a)==length(b)
        return Taylor(producto(a,b),length(b)-1)
    else
        return Taylor(producto(x,y),length(a)-1)
    end
end
function division(a,b)#división de los coeficientes tipo Taylor para dos vectores de igual número de elementos 
    if b[1]!=zero(eltype(b))
        divab=[i=zero(eltype(a)) for i=1:length(a)]
        divab[1]=a[1]/b[1]
        for k in 2:length(a)
            l=zero(eltype(a))
            for i in 1:k-1
                l+=divab[i]*b[k-i+1]
            end
            divab[k]=(1/b[1])*(a[k]-l)
        end
        return divab
    else
        return error("b[1] debe ser distinto de cero")
    end
end
function taylordiv(a,b)#función que devuelve la división de los coeficientes tipo Taylor
    (x,y)=igualvec(a,b)
    if length(a)<length(b)
        return Taylor(division(x,y),length(b)-1)
    elseif length(a)==length(b)
        return Taylor(division(a,b),length(b)-1)
    else
        return Taylor(division(x,y),length(a)-1)
    end
end
for (f,f1,f2,f3) = ((:+,Taylor,Taylor,(:(taylorsum(a.v,b.v)))),
                    (:+,Taylor,Number,(:(Taylor(a.v)+Taylor(b)))),
                    (:+,Number,Taylor,(:(Taylor(a)+Taylor(b.v)))),
                    (:-,Taylor,Taylor,(:(taylorres(a.v,b.v)))),
                    (:-,Taylor,Number,(:(Taylor(a.v)-Taylor(b)))),
                    (:-,Number,Taylor,(:(Taylor(a)-Taylor(b.v)))),
                    (:*,Taylor,Taylor,(:(taylorprod(a.v,b.v)))),
                    (:*,Taylor,Number,(:(Taylor(a.v)*Taylor(b)))),
                    (:*,Number,Taylor,(:(Taylor(a)*Taylor(b.v)))),
                    (:/,Taylor,Taylor,(:(taylordiv(a.v,b.v)))),
                    (:/,Taylor,Number,(:(Taylor(a.v)/Taylor(b)))),
                    (:/,Number,Taylor,(:(Taylor(a)/Taylor(b.v)))),
                    (:(==),Taylor,Taylor,(:(a.v==b.v && a.n==b.n))),
                    (:(==),Taylor,Number,(:(Taylor(a)==Taylor(b)))),
                    (:(==),Number,Taylor,(:(Taylor(a)==Taylor(b)))), )
              
    ex = quote
        $f(a::$f1,b::$f2)=$f3
    end
    @eval $ex
end


end

AD

In [7]:
using Base.Test
using AD

In [8]:
a=[2.,3,4,5,6]
b=[1.,1,3]
c=[1.,1,3,0,0]
x=Taylor(a)
y=Taylor(b)
z=Taylor(1.0)
h=[2.,0,0]
@testset "pruebas con operaciones entre tipos Taylor"begin
    @test z==Taylor([1],0)
    @test x==Taylor([2,3,4,5,6],4)
    @test y==Taylor([1,1,3],2)
    @test z==Taylor([1],0)
    @test x+y==Taylor([3,4,7,5,6],4)
    @test 1+y==Taylor([2,1,3],2)
    @test y+1==Taylor([2,1,3],2)
    @test x-y==Taylor([1,2,1,5,6],4)
    @test y-x==Taylor([-1,-2,-1,-5,-6],4)
    @test 1-y==Taylor([0,-1,-3],2)
    @test y-1==Taylor([0,1,3],2)
    @test y*x==Taylor(producto(c,a),4)
    @test 2*y==Taylor([2,2,6],2)
    @test y*3==Taylor([3,3,9],2)
    @test y/x==Taylor(division(c,a),4)
    @test 2/y==Taylor(division(h,b),2)
    @test y/3==Taylor([1/3,1/3,1],2)
end

Test Summary:                              | Pass  Total
  pruebas con operaciones entre tipos Taylor |   17     17


Base.Test.DefaultTestSet("pruebas con operaciones entre tipos Taylor",Any[],17,false)

In [9]:
# Muestren que su código funciona con tests adecuados
workspace()
using Base.Test
include("runtest_taylor.jl")

Test Summary:                              | Pass  Total
  pruebas con operaciones entre tipos Taylor |   17     17


Base.Test.DefaultTestSet("pruebas con operaciones entre tipos Taylor",Any[],17,false)

---

## Ejercicio 2

Obtengan las relaciones de recurrencia para las funciones $L(x) = \log\big(g(x)\big)$, $P_\alpha(x) = \big(g(x)\big)^\alpha$, $S(x) = \sin\big(g(x)\big)$, $C(x) = \cos\big(g(x)\big)$ usando el mismo procedimiento que arriba. Implementen métodos adecuados para estas funciones en el módulo, actuando sobre estructuras `Taylor` e incluyan pruebas necesarias en `runtest_taylor.jl`.

**Para la función exponencial**

$$\displaystyle{E(x)=\exp(g(x))= \sum_{k=0}^\infty E_{[k]} (x-x_0)^k}\hspace{2cm}g(x)=\sum_{k=0}^\infty g_{[k]} (x-x_0)^k$$

$$\frac{dE(x)}{dx}=\sum_{k=0}^\infty kE_{[k]} (x-x_0)^{k-1}\hspace{2cm}
\frac{dg(x)}{dx}=\sum_{k=0}^\infty kg_{[k]} (x-x_0)^{k-1}$$
$$\frac{dE(x)}{dx}=\exp(g(x))\frac{dg(x)}{dx}$$

La relación de recurrencia es

\begin{equation}
E_{[k]} = \frac{1}{k} \sum_{j=0}^k j g_{[j]} \, E_{[k-j]} = 
\frac{1}{k} \sum_{j=0}^{k} (k-j) g_{[k-j]} \, E_{[j]}, \qquad k=1,2,\dots,
\end{equation}

In [49]:
workspace()
include("TaylorDiff.jl")
using AD
G=[0,1//1]#vector que representa la función g(x)=x
function Cexp(G,n)#algoritmo que cálcula los coeficientes de Taylor para la función exp(g(x))
    E=[i=zero(eltype(G)) for i=1:n+1]
    g=veclleno(G,length(E))
    E[1]=exp(g[1])
    E[2]=g[2]*E[1]
    for k in 2:n
        p=zero(eltype(G))
        for j in 0:k-1
            p+=(k-j)*g[k+1-j]*E[j+1]
        end
    E[k+1]=p/k
    end
    return Taylor(E)
end
Cexp(G,5)

AD.Taylor{Rational{Int64}}(Rational{Int64}[1//1,1//1,1//2,1//6,1//24,1//120],5)

In [48]:
2

2

In [50]:
import Base:exp
exp(a::Taylor, b::Int)=Cexp(a.v,b)

exp (generic function with 13 methods)

In [51]:
G=[0,1//1]
a=Taylor(G)
exp(a,4)

AD.Taylor{Rational{Int64}}(Rational{Int64}[1//1,1//1,1//2,1//6,1//24],4)

**Para el logaritmo** 

$$\displaystyle{L(x)=\log(g(x))= \sum_{k=0}^\infty L_{[k]} (x-x_0)^k}\hspace{2cm}g(x)=\sum_{k=0}^\infty g_{[k]} (x-x_0)^k$$

$$\frac{dL(x)}{dx}=\sum_{k=0}^\infty kL_{[k]} (x-x_0)^{k-1}\hspace{2cm}
\frac{dg(x)}{dx}=\sum_{k=0}^\infty kg_{[k]} (x-x_0)^{k-1}$$
$$\frac{dL(x)}{dx}=\frac{d\log(g(x))}{dx}=\frac{1}{g(x)}\frac{dg(x)}{dx}$$

$$g(x)\frac{dL(x)}{dx}=\frac{dg(x)}{dx}$$

Entonces
$$\sum_{k=0}^\infty g_{[k]} (x-x_0)^k\sum_{j=0}^\infty jL_{[j]} (x-x_0)^{j-1}=\sum_{k=0}^\infty kg_{[k]} (x-x_0)^{k-1}$$

$$\left[\sum_{k=0}^\infty kL_{[k]} (x-x_0)^{k-1}\right]\left[\sum_{j=0}^\infty g_{[j]} (x-x_0)^j\right]=\sum_{k=0}^\infty kg_{[k]} (x-x_0)^{k-1}$$

$$\sum_{k=1}^\infty\left[\sum_{j=0}^k(k-j)L_{[k-j]}g_{[j]} \right] (x-x_0)^{k-1}=\sum_{k=0}^\infty kg_{[k]} (x-x_0)^{k-1}$$

$$\sum_{k=1}^\infty\left[\sum_{j=0}^kjL_{[j]}g_{[k-j]} \right] (x-x_0)^{k-1}=\sum_{k=0}^\infty kg_{[k]} (x-x_0)^{k-1}$$

$$\sum_{j=0}^kjL_{[j]}g_{[k-j]}=kg_{[k]}$$

$$kL_{[k]}g_{[0]}+\sum_{j=0}^{k-1}jL_{[j]}g_{[k-j]}=kg_{[k]}$$

$$L_{[k]}=\frac{1}{g_{[0]}}\left[g_{[k]}-\frac{1}{k}\sum_{j=1}^{k-1}jL_{[j]}g_{[k-j]}\right]$$

In [52]:
G=[1//1,1]#función g(x)=1+x
function coeflog(G,n)#algoritmo que cálcula los coeficientes de Taylor para la función log(g(x))
    L=[i=zero(eltype(G)) for i=1:n+1]
    g=veclleno(G,length(L))
    L[1]=log(g[1])
    L[2]=g[2]/g[1]
    for k in 2:n
    l=zero(eltype(G))
        for j in 1:k-1
            l+=j*L[j+1]*g[k+1-j]
        end
    L[k+1]=(1/g[1])*(g[k+1]-l/k)
    end
    return Taylor(L)
end
coeflog(G,3)

AD.Taylor{Rational{Int64}}(Rational{Int64}[0//1,1//1,-1//2,1//3],3)

In [53]:
import Base:log
log(a::Taylor, b::Int)=coeflog(a.v,b)

log (generic function with 20 methods)

In [54]:
G=[1,1//1]
a=Taylor(G)
log(a,4)

AD.Taylor{Rational{Int64}}(Rational{Int64}[0//1,1//1,-1//2,1//3,-1//4],4)

**Para la función $P_\alpha(x)=(g(x))^\alpha$** 

$$\displaystyle{P_\alpha(x)=(g(x))^\alpha= \sum_{k=0}^\infty {P_\alpha}_{[k]} (x-x_0)^k}\hspace{2cm}g(x)=\sum_{k=0}^\infty g_{[k]} (x-x_0)^k$$

$$\frac{dP_\alpha(x)}{dx}=\sum_{k=0}^\infty k{P_\alpha}_{[k]} (x-x_0)^{k-1}\hspace{2cm}
\frac{dg(x)}{dx}=\sum_{k=0}^\infty kg_{[k]} (x-x_0)^{k-1}$$
$$\frac{d(g(x))^\alpha}{dx}=\alpha (g(x))^{\alpha-1}\frac{dg(x)}{dx}=\alpha \frac{(g(x))^{\alpha}}{g(x)}\frac{dg(x)}{dx}$$

Por una parte
$$g(x)\frac{d(g(x))^\alpha}{dx}=\alpha(g(x))^{\alpha}\frac{dg(x)}{dx}$$

$$\sum_{k=0}^\infty g_{[k]} (x-x_0)^k\sum_{j=0}^\infty j{P_\alpha}_{[j]} (x-x_0)^{j-1}=
\alpha \sum_{j=0}^\infty {P_\alpha}_{[j]} (x-x_0)^j\sum_{k=0}^\infty kg_{[k]} (x-x_0)^{k-1}$$

$$\sum_{k=1}^\infty\left[\sum_{j=0}^kg_{[k-j]}j{P_\alpha}_{[j]}\right](x-x_0)^{k-1}=
\sum_{k=1}^\infty\left[\sum_{j=0}^k \alpha(k-j)g_{[k-j]}{P_\alpha}_{[j]}\right](x-x_0)^{k-1}$$

$$\sum_{j=0}^kg_{[k-j]}j{P_\alpha}_{[j]}=\sum_{j=0}^k \alpha(k-j)g_{[k-j]}{P_\alpha}_{[j]}$$

$$\sum_{j=1}^kg_{[k-j]}j{P_\alpha}_{[j]}=\sum_{j=0}^{k-1} \alpha(k-j)g_{[k-j]}{P_\alpha}_{[j]}$$

$$g_{[0]}k{P_\alpha}_{[k]}+\sum_{j=1}^{k-1}g_{[k-j]}j{P_\alpha}_{[j]}=\alpha kg_{[k]}{P_\alpha}_{[0]}+\sum_{j=1}^{k-1} \alpha(k-j)g_{[k-j]}{P_\alpha}_{[j]}$$

$$g_{[0]}k{P_\alpha}_{[k]}=\alpha kg_{[k]}{P_\alpha}_{[0]}+\sum_{j=1}^{k-1}g_{[k-j]}{P_\alpha}_{[j]}\left[\alpha(k-j)-j\right]$$

$${P_\alpha}_{[k]}=\alpha \frac{g_{[k]}}{g_{[0]}}{P_\alpha}_{[0]}+\frac{1}{kg_{[0]}}\sum_{j=1}^{k-1}g_{[k-j]}{P_\alpha}_{[j]}\left[\alpha k-(\alpha +1)j\right]$$

In [58]:
G=[1.0,1]#vector que representa la función g(x)=1+x
function Cexpalpha(G,alpha,n)#algoritmo que cálcula los coeficientes de Taylor para la función (g(x))^α
    P=[i=zero(eltype(G)) for i=1:n+1]
    g=veclleno(G,length(P))
    P[1]=(g[1])^alpha
    P[2]=alpha*(g[2]/g[1])*P[1]
    for k in 2:n
    p=zero(eltype(G))
        for j in 1:k-1
            p+=(alpha*k-(alpha+1)*j)*g[k+1-j]*P[j+1]
        end
    P[k+1]=alpha*(g[k+1]/g[1])*P[1]+(1/(k*g[1]))*p
    end
    return Taylor(P)
end
Cexpalpha(G,2,7)

AD.Taylor{Float64}([1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0],7)

In [59]:
import Base:^
^(a::Taylor,b::Int,c::Int)=Cexpalpha(a.v,b,c)

^ (generic function with 48 methods)

In [60]:
G=[1,1//1]
a=Taylor(G)
^(a,4,4)

AD.Taylor{Rational{Int64}}(Rational{Int64}[1//1,4//1,6//1,4//1,1//1],4)

**Para la función $S(x)=\sin(g(x))$ y $C(x) = \cos(g(x))$** 

$$S(x)=sin(g(x))= \sum_{k=0}^\infty {S}_{[k]} (x-x_0)^k\hspace{2cm}C(x) = \cos(g(x))=\sum_{k=0}^\infty C_{[k]} (x-x_0)^k$$

$$\frac{dS(x)}{dx}=\sum_{k=0}^\infty k{S}_{[k]} (x-x_0)^{k-1}\hspace{2cm}
\frac{dC(x)}{dx}=\sum_{k=0}^\infty k{C}_{[k]} (x-x_0)^{k-1}$$
$$\frac{dS(x)}{dx}=\cos(g(x))\frac{dg(x)}{dx}=C(x)\frac{dg(x)}{dx}\hspace{2cm}
\frac{dC(x)}{dx}=-\sin(g(x))\frac{dg(x)}{dx}=-S(x)\frac{dg(x)}{dx}$$

Igualando las últimas 4 expresiones
$$\frac{dS(x)}{dx}=\sum_{k=0}^\infty k{S}_{[k]} (x-x_0)^{k-1}=
\sum_{k=0}^\infty C_{[k]} (x-x_0)^k\sum_{j=0}^\infty jg_{[j]} (x-x_0)^{j-1}=
\sum_{k=1}^\infty \left[\sum_{j=0}^k jg_{[j]}C_{[k-j]}\right](x-x_0)^{k-1}
$$
$$\frac{dC(x)}{dx}=\sum_{k=0}^\infty k{C}_{[k]} (x-x_0)^{k-1}=
-\sum_{k=0}^\infty S_{[k]} (x-x_0)^k\sum_{j=0}^\infty jg_{[j]} (x-x_0)^{j-1}=
-\sum_{k=1}^\infty \left[\sum_{j=0}^k jg_{[j]}S_{[k-j]}\right](x-x_0)^{k-1}
$$

De aquí obtengo las dos siguientes relaciones

$$S_{[k]}=\frac{1}{k}\sum_{j=0}^k jg_{[j]}C_{[k-j]}=\frac{1}{k}\sum_{j=0}^{k-1} (k-j)g_{[k-j]}C_{[j]}\hspace{2cm}
C_{[k]}=-\frac{1}{k}\sum_{j=0}^k jg_{[j]}S_{[k-j]}=-\frac{1}{k}\sum_{j=0}^{k-1} (k-j)g_{[k-j]}S_{[j]}$$

Con las condiciones iniciales 
$$S_{[0]}=\sin(g(x_0))\hspace{2cm}C_{[0]}=\cos(g(x_0))$$

Para el caso concreto $g(x) = x$ alrededor de $x_0=0$, donde tenemos $g_{[j]} = \delta_{j,1}$, obtenemos

$$S_{[0]}=0 \hspace{2cm}C_{[0]}=1$$
$$S_{[1]}=C_{[0]}=1 \hspace{2cm}C_{[1]}=-S_{[0]}=0$$

$$S_{[k]}=\frac{1}{k}C_{[k-1]}=\frac{1(-1)}{k(k-1)}S_{[k-2]}=\frac{1(-1)(1)}{k(k-1)(k-2)}C_{[k-3]}=
\frac{1(-1)(1)(-1)}{k(k-1)(k-2)(k-3)}S_{[k-4]}=\dots$$$$
C_{[k]}=\frac{-1}{k}S_{[k-1]}=\frac{(-1)(1)}{k(k-1)}C_{[k-2]}=\frac{(-1)(1)(-1)}{k(k-1)(k-2)}S_{[k-3]}=
\frac{(-1)(1)(-1)(1)}{k(k-1)(k-2)(k-3)}C_{[k-4]}=\dots$$

$$S_{[k]}=\frac{(-1)}{k(k-1)}S_{[k-2]}=\frac{(-1)^2}{k(k-1)(k-2)(k-3)}S_{[k-4]}=
\frac{(-1)^3}{k(k-1)(k-2)(k-3)(k-4)(k-5)}S_{[k-6]}=\dots$$$$
C_{[k]}=\frac{(-1)}{k(k-1)}C_{[k-2]}=\frac{(-1)^2}{k(k-1)(k-2)(k-3)}C_{[k-4]}=
\frac{(-1)^3}{k(k-1)(k-2)(k-3)(k-4)(k-5)}C_{[k-6]}=\dots$$

$$S_{[k]}=\frac{(-1)^{j/2}(k-j)!}{k!}S_{[k-j]}\hspace{2cm}C_{[k]}=\frac{(-1)^{j/2}(k-j)!}{k!}C_{[k-j]}$$

Para que $S_{[k]}$ no sea nulo $k>2$ y debe ser un número impar $k=2n+1$. De la misma menera para que $C_{[k]}$ no sea nulo $k\geq2$ y debe ser un número par $k=2n$. Entonces

$$S_{[2n+1]}=\frac{(-1)^{j/2}((2n+1)-j)!}{(2n+1)!}S_{[(2n+1)-j]}\hspace{2cm}C_{[2n]}=\frac{(-1)^{j/2}((2n)-j)!}{(2n)!}C_{[(2n)-j]}$$

Para los coeficientes del seno si $2n+1-j=1$ $\Rightarrow j=2n$. En los coeficientes del coseno si $2n-j=0$ $\Rightarrow j=2n$. De esta manera

$$S_{[2n+1]}=\frac{(-1)^{n}}{(2n+1)!}S_{[1]}=\frac{(-1)^{n}}{(2n+1)!}C_{[0]}=\frac{(-1)^{n}}{(2n+1)!}\hspace{2cm}C_{[2n]}=\frac{(-1)^{n}}{(2n)!}C_{[0]}=\frac{(-1)^{n}}{(2n)!}$$

Por lo tanto
$$S(x)=\sin(x)=\sum_{n=0}^\infty \frac{(-1)^{n}}{(2n+1)!}x^{2n+1}\hspace{2cm}C(x) = \cos(x)=\sum_{n=0}^\infty \frac{(-1)^{n}}{(2n)!}x^{2n}$$

In [61]:
G=[0,1.0]#vector que representa la función g(x)=x
function Csencos(G,n)#algoritmo que cálcula los coeficientes de Taylor para las funciones cos(g(x)) y sin(g(x))
    S=[i=zero(eltype(G)) for i=1:n+1]
    C=[i=zero(eltype(G)) for i=1:n+1]
    g=veclleno(G,length(S))
    C[1]=cos(g[1])
    S[2]=C[1]
    for k in 2:n
        p=zero(eltype(G))
        q=zero(eltype(G))
        for j in 0:k-1
            p+=(k-j)*g[k+1-j]*C[j+1]
            q+=(k-j)*g[k+1-j]*S[j+1]
        end
    S[k+1]=p/k
    C[k+1]=-q/k
    end
    return Taylor(C),Taylor(S)
end
function Csen(G,n)
    (a,b)=Csencos(G,n)
    return b
end
function Ccos(G,n)
    (a,b)=Csencos(G,n)
    return a
end
Ccos(G,4)

AD.Taylor{Float64}([1.0,0.0,-0.5,-0.0,0.0416667],4)

In [64]:
import Base:sin,cos
sin(a::Taylor,b::Int)=Csen(a.v,b)
cos(a::Taylor,b::Int)=Ccos(a.v,b)

cos (generic function with 11 methods)

In [65]:
G=[0,1//1]
a=Taylor(G)
sin(a,4),cos(a,4)

(AD.Taylor{Rational{Int64}}(Rational{Int64}[0//1,1//1,0//1,-1//6,0//1],4),AD.Taylor{Rational{Int64}}(Rational{Int64}[1//1,0//1,-1//2,0//1,1//24],4))

In [75]:
a=[0,1.]
b=[1,1.]
x=Taylor(a)
y=Taylor(b)

using Base.Test
@testset "pruebas con operaciones entre funciones de tipos Taylor"begin
    @test exp(x,3)==Cexp(a,3)
    @test exp(x,6)==Cexp(a,6)
    @test log(y,3)==coeflog(b,3)
    @test log(y,5)==coeflog(b,5)
    @test ^(y,5,5)==Cexpalpha(b,5,5)
    @test ^(y,5,3)==Cexpalpha(b,5,3)
    @test sin(x,3)==Csen(a,3)
    @test sin(x,5)==Csen(a,5)
    @test cos(x,5)==Ccos(a,5)
    @test cos(x,5)==Ccos(a,5)
end

Test Summary:                                           | Pass  Total
  pruebas con operaciones entre funciones de tipos Taylor |   10     10


Base.Test.DefaultTestSet("pruebas con operaciones entre funciones de tipos Taylor",Any[],10,false)

Aquí incluyo las pruebas finales.

In [77]:
workspace()
using Base.Test
include("runtest_taylor2.jl")

Test Summary:                              | Pass  Total
  pruebas con operaciones entre tipos Taylor |   17     17
Test Summary:                                           | 

Pass  Total
  pruebas con operaciones entre funciones de tipos Taylor |   10     10


Base.Test.DefaultTestSet("pruebas con operaciones entre funciones de tipos Taylor",Any[],10,false)